In [1]:
import pandas as pd
import numpy as np
import json
import logging
import sys
import spacy
import re
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.gold import GoldParse
import en_core_web_sm

In [90]:
big_list = []
for i in range(1,18):
    name = 'data/'+str(i)+'.txt'
    damp = open(name).read().split('\n')
    #damp = [elem  for elem in damp if len(elem)>0]
    big_list.extend(damp)
    

In [209]:
# list_2 = open("1.txt").read().split('\n')
# list_2 = [elem  for elem in list_2 if len(elem)>0]
# len(list_2)
len(big_list)

16411

In [19]:
tag_list = ['CVE','CWE','SOFTWARE','MALWARE','COURSE_OF_ACTION',
           'INTRUSION_SET','THREAT_ACTOR','TOOL','ATTACK_PATTERN','MITRE_ATTACK',
            'INDUSTRY','TIMESTAMP','CAMPAIGN','ORG','COUNTRY','CITY',
            'GEOLOCATION','IOC'
           ]

In [191]:
spacy_train2 = []
for elem in big_list[:2000]:
    if '(<' or '>)' in elem:
        elem = elem.replace('(<','<')
        elem = elem.replace('>)','>')
    for tag in tag_list:
        tag_start = '<' + tag +'>'
        if tag_start in elem:
            annotation = {}
            end_tag = '</'+ tag + '>'
            label_list = re.findall(tag_start+'(.*?)'+end_tag,elem)
            if len(label_list) == 1:
                pos_start = elem.find(label_list[0])
                pos_end = elem.find(label_list[0]) + len(label_list[0])
                ent_dict = {}
                ent_dict['entities'] = [(pos_start+1,pos_end+1,tag)]
                #elem = elem[0:pos_end] + ' ' + elem[pos_end:]
                elem = elem[0:pos_start] +' ' +  label_list[0] + ' ' + elem[pos_end:]
                spacy_sent = (elem,ent_dict)
                spacy_train2.append(spacy_sent)
               
            else:
                ent_dict = {}
                ent_dict['entities'] = []
                for mult_label in label_list:
                        pos_start = elem.find(mult_label)
                        pos_end = elem.find(mult_label) + len(mult_label)
                        ent_dict['entities'].append((pos_start+1,pos_end+1,tag))
                        #elem = elem[0:pos_end] + ' ' + elem[pos_end:]
                        elem = elem[0:pos_start] +' ' +  mult_label + ' ' + elem[pos_end:]
                spacy_train2.append((elem,ent_dict))

In [196]:
# thinc_gpu_ops.AVAILABLE
# spacy.prefer_gpu()
nlp_test=spacy.blank("en")
nlp_test.add_pipe(nlp_test.create_pipe('ner'))
nlp_test.begin_training()
ner_test=nlp_test.get_pipe("ner")
TRAIN_DATA = spacy_train2

In [197]:
for _, annotations in TRAIN_DATA:
      for ent in annotations.get("entities"):
        ner_test.add_label(ent[2])

In [208]:
len(TRAIN_DATA)

392

In [198]:
optimizer = nlp_test.resume_training()
for itn in range(20):
# shuffle examples before training
    random.shuffle(TRAIN_DATA)
    # batch up the examples using spaCy's minibatch
    # ictionary to store losses
    losses = {}
    for text,ent in TRAIN_DATA:
        content = text
        annot = ent['entities']
        #print(text,annot)
        text = nlp_test.make_doc(content)
        if len(annot) != 0:
            cust_ann = [(annot[0][0],annot[0][1],annot[0][2])] 
            gold = GoldParse(text,entities = cust_ann)
            nlp_test.update([text],[gold],sgd=optimizer,losses=losses)

/Users/vlad3d/opt/anaconda3/envs/TensorFlow2/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: [W030] Some entities could not be aligned in the text "<SOFTWARE>  Java Script </SOFTWARE> objects were n..." with entities "[(11, 21, 'SOFTWARE')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  from ipykernel import kernelapp as app
/Users/vlad3d/opt/anaconda3/envs/TensorFlow2/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: [W030] Some entities could not be aligned in the text "<SOFTWARE>  Java  </SOFTWARE> Version <= 1.6.0_27 ..." with entities "[(11, 15, 'SOFTWARE')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  from ipykernel import kernelapp as app
/Users/vlad3d/opt/anaconda3/envs/TensorFlow2/lib/python3

In [231]:
tester = open("test_3.txt").read().split('\n')
#tester = [elem  for elem in tester  if len(elem)>0]

In [232]:
teg_list = []
for sent in tester:
    doc = nlp_test(sent)
    if len(doc.ents) != 0:
        for elem in doc.ents:
            teg_list.append((elem.text,elem.label_))

In [233]:
teg_list

[('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Emergency', 'SOFTWARE'),
 ('Report Table', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Emergency', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Remote Desktop Protocol)', 'MALWARE'),
 ('Operation Ghost', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('. Since', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Info of', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('2nd Comparison', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('winsec64 Decryption', 'SOFTWARE'),
 ('Time-', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security Trend', 'SOFTWARE'),
 ('Security 

In [72]:
import pickle
with open('list_f4.pkl', 'wb') as fp:
    pickle.dump(teg_list, fp)

In [222]:
output_dir = Path('models/')
nlp_test.to_disk(output_dir)

In [223]:
print("Loading from Dir", output_dir)
nlp_loaded = spacy.load(output_dir)

Loading from Dir models


In [229]:
preds = open("test_4.txt").read().split('\n')
preds = [elem  for elem in preds  if len(elem)>0]
pred_list = []
for sent in preds:
    doc = nlp_loaded(sent)
    if len(doc.ents) != 0:
        for elem in doc.ents:
            pred_list.append((elem.text,elem.label_))

In [230]:
pred_list[:5]

[('SPECIAL', 'SOFTWARE'),
 ('SPECIAL', 'SOFTWARE'),
 ('SPECIAL', 'SOFTWARE'),
 ('SPECIAL REPORT', 'SOFTWARE'),
 ('M-TRENDS 2020', 'TIMESTAMP')]

In [87]:
nlp_test.vocab.vectors_length

0